<div class="alert alert-block alert-info"> <b>NOTE</b> 

<div>
    This is a duplicate of notebook <b>04_prediction-best-practices </b>, considering the new performance metric adjustments the methodolgy to evaluate the test data changes to a sequential approach. Each time series is evaluated independently instead of the concatenations done before.
</div>
</div>

In [ ]:
from auxiliary_functions import load_joblib
from evaluation_metrics import  time_window_perform, oscillation_red, class_report

# For testing data

<div class="alert alert-block alert-danger"> <b>NOTE</b> 

<div>
    Choose the working_class correctly. 1 for blocking, 0 for non-blocking.
    Additionally pay attention to the lag value.
</div>
</div>

In [2]:
#Load the model and data
lag = 60
path_to_read1 = f'/Users/luisescobar/Documents/Thesis/Models/original/lag_{lag}min'
path_to_read2 = '/Users/luisescobar/Documents/Thesis/DataSets/Dictionary/03_01_Train_Val_Test'
#path_to_read3 = '/Users/luisescobar/Documents/Thesis/Models'
filename_model = 'xgboost_model.pkl'
filename_clot = 'blocking_20.pkl'
filename_noclot = 'no_blocking_20.pkl'
filename_scaler = 'scaler.pkl'

#model_loaded = load_joblib(path_to_read1, filename_model)
model_loaded = load_joblib(path_to_read1, filename_model)
scaler_loaded = load_joblib(path_to_read1, filename_scaler)
clot_dict = load_joblib(path_to_read2, filename_clot)
no_clot_dict = load_joblib(path_to_read2, filename_noclot)

working_class = 1 #must be 1 (clot)/ 0 (non-clot)

In [3]:
#Choose arbitrary a dataset from the blocking class
if working_class == 1:
    print(clot_dict.keys())
    dict_eval = clot_dict
elif working_class == 0:
    print(no_clot_dict.keys())
    dict_eval = no_clot_dict

dict_keys(['df_1', 'df_923', 'df_680-df_681-concat', 'df_654', 'df_311', 'df_344-df_345-df_346-concat', 'df_902', 'df_486', 'df_513', 'df_62', 'df_653', 'df_783', 'df_790', 'df_247', 'df_190', 'df_751', 'df_764', 'df_859', 'df_115'])


In [4]:
use_threshold = False

data_p0 = []
data_r0 = []
data_p1 = []
data_r1 = []

for df_name in dict_eval.keys():
    #print(df_name)
    df = dict_eval[df_name]
    
    
    
    # Create lagged features for each column except the target column
    for column in df.columns:
        if column != 'Clotting_2':  # Skip the target column
            df[f'{column}_lag_{lag}'] = df[column].shift(lag)
    
    # Remove rows with NaN values (due to shifting)
    df_lagged = df.dropna()
    df_lagged = df_lagged.reset_index(drop=True)
    
    # Prepare features (X) and target (y)
    # Drop original columns and only use lagged features
    lagged_columns = [col for col in df_lagged.columns if 'lag_' in col]
    X = df_lagged[lagged_columns]
    X = scaler_loaded.transform(X)
    y_true = df_lagged['Clotting_2'] 
    
    
    
    if use_threshold is False:
        # Make predictions using the lag feature matrix
        y_pred = model_loaded.predict(X)
        
    else:
        # Get predicted probabilities
        probs = model_loaded.predict_proba(X)
        
        # Define a custom threshold, e.g., 0.7
        threshold = 0.7
        
        # Assign Class 1 if the probability for Class 1 is greater than or equal to the threshold, else Class 0
        y_pred = (probs[:, 1] >= threshold).astype(int)
    
    # Regularization on oscillations (on test)
    y_pred_reg = oscillation_red(y_pred, 0.4, 20)
    
    
    p0, r0, p1, r1 =  time_window_perform(y_pred_reg, y_true, 20, False)
    
    data_p0.append(p0)
    data_r0.append(r0)
    data_p1.append(p1)
    data_r1.append(r1)

In [5]:
class_report(data_p0, data_r0, data_p1, data_r1, 'mean')

Classification Report:
+-----+-------------+----------+
|     |   precision |   recall |
+=====+=============+==========+
|   0 |    0.921763 | 0.970503 |
+-----+-------------+----------+
|   1 |    0.285525 | 0.328878 |
+-----+-------------+----------+


In [6]:
class_report(data_p0, data_r0, data_p1, data_r1, 'median')

Classification Report:
+-----+-------------+----------+
|     |   precision |   recall |
+=====+=============+==========+
|   0 |    0.970588 |        1 |
+-----+-------------+----------+
|   1 |    0        |        0 |
+-----+-------------+----------+
